### CM1 horizontal transport in y 

In [1]:
import sys
import os
import numpy as np
import xarray as xr
from glob import glob
import matplotlib.pyplot as plt
import warnings
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle
import metpy.calc as mpc
from metpy.units import units
# for animations
from PIL import Image

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def read_data(data_dir,t_start=-144):
    os.chdir(data_dir)
    print(data_dir)
    file_name = glob('*nc')
    data_zon = xr.open_dataset('cm1out_zonmean.nc')
    data_3d = xr.open_dataset('cm1out_sub3d.nc')
    data_2d = xr.open_dataset('cm1out_2Dvars.nc')
    
    # 3d data
    th3d = data_3d.th[t_start:,:45,:,:]    
    u3d = data_3d.uinterp[t_start:,:45,:,:]
    v3d = data_3d.vinterp[t_start:,:45,:,:]
    w3d = data_3d.winterp[t_start:,:45,:,:]
    qv3d = data_3d.qv[t_start:,:45,:,:]

    # zonal mean data
    th = data_zon.th[t_start:,:,:,:]
    qv = data_zon.qv[t_start:,:,:,:]
    u = data_zon.uinterp[t_start:,:,:,:]
    v = data_zon.vinterp[t_start:,:,:,:]
    w = data_zon.winterp[t_start:,:,:,:]
    qc = data_zon.qc[t_start:,:,:,:]
    qi = data_zon.qi[t_start:,:,:,:]
    prs = data_zon.prs[t_start:,:,:,:]
    # 2d data
    prate = data_2d.prate[t_start:,:,:]
    pwat = data_2d.pwat[t_start:,:,:]
    cwp = data_2d.cwp[t_start:,:,:]
    cape = data_2d.cape[t_start:,:,:]
    cin = data_2d.cin[t_start:,:,:]
    
    # temperature conversion
    T = th/((1000*100/prs)**(287.15/1004))-273.15 # [degC]
    # density 
    ro = prs/(287.15*(T+273.15))

    x_dim = data_2d.lon
    y_dim = data_2d.lat
    div = np.gradient(v,y_dim*1000,axis=2) # du/dx+dv/dy 
    vor = -np.gradient(u,y_dim*1000,axis=2) # -du/dy+dv/dx
    #relative humidty calculation, Buck (1996)
    es_liq = 0.61121*np.exp((18.678-T/234.5)*(T/(257.14+T)))*10 #[hpa]
    es_ice = 0.61115*np.exp((23.036-T/333.7)*(T/(279.82+T)))*10

    qs_liq = 0.622*es_liq/(prs/100-es_liq)
    qs_ice = 0.622*es_ice/(prs/100-es_ice)

    rh = qv/qs_liq
    rh_ice = qv/qs_ice

    rh = rh.values.flatten()
    rh_ice = rh_ice.values.flatten()
    T_test = T.values.flatten()

    rh[T_test<0] = rh_ice[T_test<0]
    rh = rh.reshape((T.shape[0],T.shape[1],T.shape[2],T.shape[3]))
    rh = xr.DataArray(rh,coords=[T.time,T.lev,T.lat,T.lon],dims=['time','lev','lat','lon'])
    
    return (th,T,qv,qc,qi,rh,prate*3600*24,pwat*1000,cwp,u,v,w,div,vor,cape,cin,prs,ro,th3d,u3d,v3d,w3d,qv3d)

In [ ]:
# get weather or weather2 
server = '/data2/willytsai/cm1r19.8/analysis/runs_cheyenne'
#server = '/w2-data2/willytsai/cm1r19.8/analysis/runs_cheyenne'
os.chdir(server)

exp_name = ['CM1_RAD4K_RCEP_240x240_U204km',
            'CM1_RAD4K_RCEP_240x240_lowU05',
            'CM1_RAD4K_RCEP_240x240_lowU07',
            'CM1_RAD4K_RCEP_240x240_lowU10',
            'CM1_RAD4K_RCEP_240x240_lowU12',
            'CM1_RAD4K_RCEP_240x240_lowU15']

exp_short = ['U20_4km','U05','U07','U10','U12','U15']
print(exp_name)
print('Number of Exp: ', len(exp_name))
fig_dir = server + '/fig_sum'

In [ ]:
(th1,T1,qv1,qc1,qi1,rh1,prate1,pwat1,cwp1,u1,v1,w1,div1,vor1,cape1,cin1,prs1,ro1,th1_3d,u1_3d,v1_3d,w1_3d,qv1_3d) = read_data(server+'/'+exp_name[0])
(th2,T2,qv2,qc2,qi2,rh2,prate2,pwat2,cwp2,u2,v2,w2,div2,vor2,cape2,cin2,prs2,ro2,th2_3d,u2_3d,v2_3d,w2_3d,qv2_3d) = read_data(server+'/'+exp_name[1])
(th3,T3,qv3,qc3,qi3,rh3,prate3,pwat3,cwp3,u3,v3,w3,div3,vor3,cape3,cin3,prs3,ro3,th3_3d,u3_3d,v3_3d,w3_3d,qv3_3d) = read_data(server+'/'+exp_name[2])
(th4,T4,qv4,qc4,qi4,rh4,prate4,pwat4,cwp4,u4,v4,w4,div4,vor4,cape4,cin4,prs4,ro4,th4_3d,u4_3d,v4_3d,w4_3d,qv4_3d) = read_data(server+'/'+exp_name[3])
(th5,T5,qv5,qc5,qi5,rh5,prate5,pwat5,cwp5,u5,v5,w5,div5,vor5,cape5,cin5,prs5,ro5,th5_3d,u5_3d,v5_3d,w5_3d,qv5_3d) = read_data(server+'/'+exp_name[4])
(th6,T6,qv6,qc6,qi6,rh6,prate6,pwat6,cwp6,u6,v6,w6,div6,vor6,cape6,cin6,prs6,ro6,th6_3d,u6_3d,v6_3d,w6_3d,qv6_3d) = read_data(server+'/'+exp_name[5])

In [ ]:
# horizontal transport 
v_tmp = v3_3d
u_tmp = u3_3d
qv_tmp = qv3_3d
y_dim = v3_3d.lon.values
x_dim = v3_3d.lat.values
ro_tmp = ro3[0,:45,:,0].mean(axis=1) # ro profile
qv_div = (ro_tmp*v_tmp*qv_tmp).differentiate('lat')/1000 \
         + (ro_tmp*u_tmp*qv_tmp).differentiate('lon')/1000 # vapor mass flux divergence 

In [ ]:
# using pressure coordinate 
p_dim = prs1[0,:,:,0].mean(axis=1) # 65 p-levels

In [ ]:
z_dim = v1_3d.lev
y_dim = v1_3d.lat
strmf_ctrl = np.zeros((144,len(z_dim),len(y_dim)))

for n,(ro,v3d) in enumerate(zip([ro1],[v1_3d])):
    v = v3d.mean('lon')
    for t in range(144):
        for z in range(len(z_dim)):
            strmf_ctrl[t,z,:] = np.trapz(ro[t,:z,:,0]*(v[t,:z,:]),z_dim[:z]*1000,axis=0) 

In [ ]:
z_dim = v3_3d.lev
y_dim = v3_3d.lat
strmf = np.zeros((len(exp_name)-1,144,len(z_dim),len(y_dim)))

for n,(ro,v3d) in enumerate(zip([ro2,ro3,ro4,ro5,ro6],[v2_3d,v3_3d,v4_3d,v5_3d,v6_3d])):
    v = v3d.mean('lon')
    for t in range(144):
        for z in range(len(z_dim)):
            strmf[n,t,z,:] = np.trapz(ro[t,:z,:,0]*(v[t,:z,:]),z_dim[:z]*1000,axis=0) 

In [ ]:
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
cmap_new = cm.get_cmap('jet',256)
newcolors = cmap_new(np.linspace(0, 1, 160))
grey_new = cm.get_cmap('Greys',256)
grey_new = grey_new(np.linspace(0,1,30))
# replace by grey colors
newcolors[:10,:] = grey_new[:10,:]
newcmap = ListedColormap(newcolors)

In [ ]:
# for CTRL only

clevs = np.linspace(-1000,1000,21) # kg/(m*s)
clab = clevs[np.abs(clevs)>0]
q_clevs = np.linspace(5*1e-6,1e-4,11)*1000 # g/kg

for n,(qc,qi) in enumerate(zip([qc1],[qi1])): # all run, except for CTRL
    fig = plt.figure(figsize=(12,4))
    strmf_re = -strmf_ctrl[:,:,:].mean(axis=0)
    #strmf_re = np.concatenate([strmf_re,strmf_re],axis=1)
    cf1 = plt.contour(y_dim,p_dim[:45]/100,strmf_re,levels=clab,colors=['k'])
    cf1.clabel(clab,fmt='%4d')

    qci_re = 1000*(qc+qi)[:,:45,:,:].squeeze().mean(axis=0)
    #qci_re = np.concatenate([qci_re, qci_re],axis=1)
    cf2 = plt.contourf(y_dim,p_dim[:45]/100,qci_re
                   ,cmap=newcmap,levels=q_clevs)
    cbar = plt.colorbar(cf2,shrink=0.7,pad=0.02)
    cbar.set_label('qc+qi [g/kg]')
    plt.xticks(np.linspace(0,480,9))
    plt.ylabel('Pressure [hPa]',fontsize=13)
    plt.xlabel('[km]',fontsize=13)
    plt.title(exp_name[n])
    plt.ylim([1000,50])

    plt.close()
    fig.savefig(fig_dir+'/streamfunc_qcqi_'+exp_short[n]+'.pdf',
              bbox_inches='tight',transparent=False)

In [ ]:
# mass vapor plot
# for CTRL only

clevs = np.linspace(-1000,1000,21) # kg/(m*s)
clab = clevs[np.abs(clevs)>0]
q_clevs = np.linspace(-1,1,11) # g/m^3

for n,(ro,qv) in enumerate(zip([ro1],[qv1])): # all run, except for CTRL
    fig = plt.figure(figsize=(12,4))
    strmf_re = -strmf_ctrl[:,:,:].mean(axis=0)
    #strmf_re = np.concatenate([strmf_re,strmf_re],axis=1)
    cf1 = plt.contour(y_dim,p_dim[:45]/100,strmf_re,levels=clab,colors=['k'])
    cf1.clabel(clab,fmt='%4d')

    qv_re = 1000*(ro*qv)[:,:45,:,:].squeeze().mean(axis=0) # [g/m^3]
    qv_anom = qv_re - qv_re.mean(axis=1)
    cf2 = plt.contourf(y_dim,p_dim[:45]/100,qv_anom
                   ,cmap=newcmap,levels=q_clevs)
    cbar = plt.colorbar(cf2,shrink=0.7,pad=0.02)
    cbar.set_label('qv [g/m$^3$]')
    plt.xticks(np.linspace(0,480,9))
    plt.ylabel('Pressure [hPa]',fontsize=13)
    plt.xlabel('[km]',fontsize=13)
    plt.title(exp_name[n])
    plt.ylim([1000,50])

    fig.savefig(fig_dir+'/streamfunc_qv_'+exp_short[n]+'.pdf',
               bbox_inches='tight',transparent=False)

In [ ]:
clevs = np.linspace(-1000,1000,21) # kg/(m*s)
clab = clevs[np.abs(clevs)>0]
q_clevs = np.linspace(-1,1,11) # g/m^3

for n,(ro,qv) in enumerate(zip([ro2,ro3,ro4,ro5,ro6],[qv2,qv3,qv4,qv5,qv6])): # all run, except for CTRL
    fig = plt.figure(figsize=(12,4))
    cf1 = plt.contour(y_dim,p_dim[:45]/100,-strmf[n,:,:,:].mean(axis=0),levels=clab,colors=['k'])
    cf1.clabel(clab,fmt='%4d')

    qv_re = 1000*(ro*qv)[:,:45,:,:].squeeze().mean(axis=0) # [g/m^3]
    qv_anom = qv_re - qv_re.mean(axis=1)
    cf2 = plt.contourf(y_dim,p_dim[:45]/100,qv_anom
                   ,cmap=newcmap,levels=q_clevs)
    cbar = plt.colorbar(cf2,shrink=0.7,pad=0.02)
    cbar.set_label('qv [g/m$^3$]')
    plt.xticks(np.linspace(0,480,9))
    plt.ylabel('Pressure [hPa]',fontsize=13)
    plt.xlabel('[km]',fontsize=13)
    plt.title(exp_name[n+1])
    plt.ylim([1000,50])

    plt.close()
    fig.savefig(fig_dir+'/streamfunc_qv_'+exp_short[n+1]+'.pdf',
               bbox_inches='tight',transparent=False)

In [ ]:
z_dim = v3_3d.lev
y_dim = v3_3d.lat
strmf_vap = np.zeros((len(exp_name)-1,144,len(z_dim),len(y_dim))) # vapor mass streamfunc

for n,(ro,v3d,qv3d) in enumerate(zip([ro2,ro3,ro4,ro5],[v2_3d,v3_3d,v4_3d,v5_3d]
                                  ,[qv2_3d,qv3_3d,qv4_3d,qv5_3d])):
    v = v3d.mean('lon')
    qv = qv3d.mean('lon')
    for t in range(144):
        for z in range(len(z_dim)):
            strmf_vap[n,t,z,:] = np.trapz(ro[t,:z,:,0]*qv[t,:z,:]*v[t,:z,:],z_dim[:z]*1000,axis=0) 

In [ ]:
fig = plt.figure(figsize=(12,4))
cf1 = plt.contourf(y_dim,p_dim[:45]/100,-strmf_vap[3,:,:,:].mean(axis=0),levels=21,cmap='jet')
#cf1.clabel(clab,fmt='%4d')

#cf2 = plt.contourf(y_dim,p_dim[:45]/100,1000*(qc5+qi5)[:,:45,:,:].squeeze().mean(axis=0)
#               ,cmap=newcmap,levels=q_clevs)
cbar = plt.colorbar(cf1,shrink=0.7,pad=0.02)
#cbar.set_label('qc+qi [g/kg]')
plt.xticks(np.linspace(0,480,9))
plt.ylabel('Pressure [hPa]',fontsize=13)
plt.xlabel('[km]',fontsize=13)
plt.title(exp_name[4])
plt.ylim([1000,50])

In [ ]:
clevs = np.linspace(-1000,1000,21) # kg/(m*s)
clab = clevs[np.abs(clevs)>0]
q_clevs = np.linspace(5*1e-6,1e-4,11)*1000 # g/kg

for n,(qc,qi) in enumerate(zip([qc2,qc3,qc4,qc5],[qi2,qi3,qi4,qi5])): # all run, except for CTRL
    fig = plt.figure(figsize=(12,4))
    cf1 = plt.contour(y_dim,p_dim[:45]/100,-strmf[n,:,:,:].mean(axis=0),levels=clab,colors=['k'])
    cf1.clabel(clab,fmt='%4d')

    cf2 = plt.contourf(y_dim,p_dim[:45]/100,1000*(qc+qi)[:,:45,:,:].squeeze().mean(axis=0)
                   ,cmap=newcmap,levels=q_clevs)
    cbar = plt.colorbar(cf2,shrink=0.7,pad=0.02)
    cbar.set_label('qc+qi [g/kg]')
    plt.xticks(np.linspace(0,480,9))
    plt.ylabel('Pressure [hPa]',fontsize=13)
    plt.xlabel('[km]',fontsize=13)
    plt.title(exp_name[n+1])
    plt.ylim([1000,50])

    plt.close()
    fig.savefig(fig_dir+'streamfunc_qcqi_'+exp_short[n+1]+'.pdf',
               bbox_inches='tight',transparent=False)

In [ ]:
# cloud condensate difference
fig = plt.figure(figsize=(12,4))
tmp3 =1000*(qc2+qi2)[:,:45,:,:].squeeze().mean(axis=0)
tmp_ctr = 1000*(qc1+qi1)[:,:45,:,:].squeeze().mean(axis=0)
cf = plt.contourf(y_dim,v3_3d.lev,(tmp3-np.hstack((tmp_ctr,tmp_ctr)))
               ,cmap='bwr',levels=np.linspace(-0.04,0.04,17),extend='both')
cf.cmap.set_under('darkblue')
cbar = plt.colorbar(cf,shrink=0.7,pad=0.02)
cbar.set_label('$\Delta$(qc+qi) [g/kg]')
plt.xticks(np.linspace(0,480,9))
plt.ylabel('Height [km]',fontsize=13)
plt.xlabel('[km]',fontsize=13)
plt.title(exp_name[1])

In [ ]:
# convective heating
data = xr.open_dataset(server+'/'+exp_name[4]+'/cm1out_qvbgt.nc')
qt_cond = data.qt_cond

In [ ]:
fig = plt.figure(figsize=(12,4))
cf = plt.contourf(y_dim,v3_3d.lev,86400*2.5e6/1004*data.qt_evar[-144:,:45,:,:].mean(axis=(0,3))
                 ,cmap='bone_r',levels=10)
cbar = plt.colorbar(cf,shrink=0.7,pad=0.02)
cbar.set_label('Evaporation of rain [K/day]')
plt.xticks(np.linspace(0,480,9))
plt.ylabel('Height [km]',fontsize=13)
plt.xlabel('[km]',fontsize=13)
plt.ylim([0,5])

In [ ]:
fig = plt.figure(figsize=(12,4))
cf = plt.contourf(y_dim,v3_3d.lev,86400*2.5e6/1004*data.qt_evar[-144:,:45,:,:].mean(axis=(0,3))
                 ,cmap='bone_r',levels=10)
cbar = plt.colorbar(cf,shrink=0.7,pad=0.02)
cbar.set_label('Evaporation of rain [K/day]')
plt.xticks(np.linspace(0,480,9))
plt.ylabel('Height [km]',fontsize=13)
plt.xlabel('[km]',fontsize=13)
plt.ylim([0,5])